In [36]:
import gurobipy as gp

In [37]:
qtd_fabricas = 4
qtd_clientes = 9

vet_ofertas = [140, 160, 120, 190]
vet_demandas = [50, 80, 30, 50, 100, 90, 120, 70, 120]

vet_custos = [[12, 25, 39, 17, 38, 40, 8, 25, 13],
              [17, 26, 20, 25, 30, 25, 14, 20, 15],
              [35, 15, 18, 20, 12, 42, 27, 26, 19],
              [28, 30, 37, 30, 28, 36, 16, 24, 32]]

oferta_total = sum(vet_ofertas)
demanda_total = sum(vet_demandas)

In [38]:
fabricas = ["Fab_{}".format(i+1) for i in range(qtd_fabricas)]
clientes = ["Cli_{}".format(i+1) for i in range(qtd_clientes)]

ofertas = dict((fabricas[i], valor) for i, valor in enumerate(vet_ofertas))
demandas = dict((clientes[i], valor) for i, valor in enumerate(vet_demandas))

custos = dict(((fabricas[i], clientes[j]), vet_custos[i][j]) for i in range(qtd_fabricas) for j in range(qtd_clientes))


In [39]:
m = gp.Model()

x = m.addVars(fabricas, clientes,  vtype=gp.GRB.INTEGER)

m.setObjective(
    gp.quicksum(x[i, j] * custos[i, j] for i in fabricas for j in clientes),
    sense=gp.GRB.MINIMIZE
)

if oferta_total > demanda_total:
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in clientes) <= ofertas[i] for i in fabricas)
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in fabricas) == demandas[j] for j in clientes)
else:
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in clientes) == ofertas[i] for i in fabricas)
    
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in fabricas) <= demandas[j] for j in clientes)

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x9a324ff4
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective 18940.000000
Presolve time: 0.00s
Presolved: 13 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (0 binary)

Root relaxation: objective 1.008000e+04, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    10080.000000 10080.0000  0.00%     -    0s

Explored 0 nodes (13 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)



In [40]:
# Imprime o plano de transporte das fábricas
for i in fabricas:
    print("Origem:", i)
    for j in clientes:
        qtd = round(x[i, j].X)
        if qtd > 0:
            print("Transportar {} unidades para {}".format(qtd, j))
    print("")

Origem: Fab_1
Transportar 50 unidades para Cli_1
Transportar 50 unidades para Cli_4
Transportar 40 unidades para Cli_7

Origem: Fab_2
Transportar 30 unidades para Cli_3
Transportar 10 unidades para Cli_6
Transportar 120 unidades para Cli_9

Origem: Fab_3
Transportar 20 unidades para Cli_2
Transportar 100 unidades para Cli_5

Origem: Fab_4
Transportar 40 unidades para Cli_2
Transportar 80 unidades para Cli_7
Transportar 70 unidades para Cli_8



In [41]:
if oferta_total > demanda_total:
    print("As fábricas a seguir tem capacidade excedente:")
    for i in fabricas:
        sobra = round(c1[i].Slack)
        if sobra > 0:
            print("Fábrica:", i, sobra, "unidades")
elif demanda_total > oferta_total:
    print("Os clientes a seguir não tiveram toda a demanda atendida:")
    for j in clientes:
        sobra = round(c2[j].Slack)
        if sobra > 0:
            print("Cliente:", j, sobra, "unidades")
else:
    print("A oferta e demanda estão balanceadas")

Os clientes a seguir não tiveram toda a demanda atendida:
Cliente: Cli_2 20 unidades
Cliente: Cli_6 80 unidades
